# Correcting Misnamed Fieldmap Scans on Flywheel

*Project: Evolution_833922* <br>
_Author: Katja Zoner_ <br>
_Updated: 05/05/2021_ <br>

Notebook to correct the misnamed dir-AP acq-dwi fieldmap scans across the Flywheel project. The dir-AP scans were mistakenly named dir-PA at the scanner. This script identifies the dir-AP scans via the `PhaseEndodingDirection` field (for nifti files) and `PhaseEncodingDirectionPositive` field (for dicom files), then corrects the acquisition labe, `ProtocolName`, and `SeriesDescription` from `fmap-EPIdist_acq-dwi_dir-PA_seq-ABCD` to `map-EPIdist_acq-dwi_dir-AP_seq-ABCD`.

**Direction is AP if:** <br>
`PhaseEndodingDirection`:           'j-'    (nifti) <br>
`PhaseEndodingDirectionPositive`:    1      (dicom)

## 0. Import packages and setup logging.

In [4]:
import flywheel
import logging
import sys

# Set up logging
logging.basicConfig(
    stream=sys.stdout,
    level=logging.INFO,
    format='%(levelname)s: %(message)s')
logger = logging.getLogger('rename_fw_files')

## 1. Instantiate Flywheel client, get project object. 

In [5]:
# Get API_KEY from FW profile
with open('/Users/hillmann/Projects/Evolution/Scripts/process/flywheel/flywheel_api_key.txt', 'r') as f:
    API_KEY = f.read().strip()

# Get client
fw = flywheel.Client(API_KEY)
assert fw, "Your Flywheel CLI credentials aren't set!"

# Specify project
project_label = "Evolution_833922"

# Get project object
project = fw.projects.find_first('label="{}"'.format(project_label))
assert project, "Project not found!"


## 2. Some helpful functions... 

In [4]:
def get_sessions(session_labels=None):
    ''' Return a list of session objects corresponding to the 
        optionally-specified session labels. 
        Default: return all session objects in project.

    Args:
        session_labels (list of str) : List of session labels
    
    Returns:
        sessions: list of session objects
    '''

    # Get all sessions in project
    sessions = project.sessions()

    # Filter by optionally-specified session label list
    if session_labels:
        sessions = [s for s in sessions if s.label in session_labels]

    return sessions

In [5]:
def update_acq_label(acq_id, new_label):
    '''Update the acquisition label on the project container.

    Args:
        acq_id (str) : Acquisition ID
        new_label (str) : Updated Acquisition Label

    '''
    new_acq_object = flywheel.models.Acquisition(label = new_label)
    # Modifying the acquisition label here 
    fw.modify_acquisition(acquisition_id= acq_id, body = new_acq_object)

## 3. Main Program

1. Loop through sessions
2. Loop through fmap acq-dwi acquisitions
3. Identify AP scans
4. Fix 'ProtocolName' and 'SeriesDescription' in file metadata for AP scans
5. Fix acquisition label for AP acquisitions


In [6]:
## Main Program

# Specify sessions (optional)
#session_labels = ["BRAIN RESEARCH^SATTERTHWAITE"]
sessions = get_sessions()

# New label for AP acquisitions, ProtocolName, and SeriesDescription
AP_label='fmap-EPIdist_acq-dwi_dir-AP_seq-ABCD'

#countAP=0
#countPA=0

# Loop through each session
for sess in sessions:

    # Loop through fmap acq-dwi acquisitions (dir-AP or dir-PA)
    for acq in sess.acquisitions.find('label=~fmap-EPIdist_acq-dwi_dir-.._seq-ABCD'):
        
        acq = acq.reload()  # Can't load metadata without this line!!!
        hasAP = False       # Bool to indicate whether or not to rename this acquisition

        # Loop through files in acquisition
        for file in acq.files:

            # Check metadata field to determine if scan direction is AP
            # Dicom scans have 'PhaseEncodingDirectionPositive' field instead of 'PhaseEncodingDirection' field
            try:
                isAP = (file.info['PhaseEncodingDirection'] == 'j-')
            except KeyError:
                isAP = (file.info['CSAHeader']['PhaseEncodingDirectionPositive'] == 1)
            
            # If AP scan -> correct protocol name and series description in file metadata 
            if isAP:
                #countAP+=1
                hasAP = True
                logger.info(f"Correcting metadata for scan {file.name} in session {sess.label}")
                file.update_info({'ProtocolName': AP_label, 'SeriesDescription': AP_label})
            
            # If PA scan -> update count
            #else:
                #countPA+=1
        
        # If acquisition consists of AP scans -> update acquisition label
        if hasAP:
            logger.info(f"Renaming acquisition {acq.id} in session {sess.label}")
            update_acq_label(acq.id, AP_label)


#print(countAP)
#print(countPA)


INFO: Correcting metadata for scan 1.3.12.2.1107.5.2.43.167024.2020020412555417013054399.0.0.0.dicom.zip in session BRAIN RESEARCH^SATTERTHWAITE
INFO: Correcting metadata for scan 1_3_12_2_1107_5_2_43_167024_2020020412555417013054399_0_0_0.json in session BRAIN RESEARCH^SATTERTHWAITE
INFO: Correcting metadata for scan 1_3_12_2_1107_5_2_43_167024_2020020412555417013054399_0_0_0.nii.gz in session BRAIN RESEARCH^SATTERTHWAITE
INFO: Renaming acquisition 5e39b08e6dea316b862a746b in session BRAIN RESEARCH^SATTERTHWAITE
INFO: Correcting metadata for scan 1.3.12.2.1107.5.2.43.167024.2020021412403753831414624.0.0.0.dicom.zip in session 11472
INFO: Correcting metadata for scan 1_3_12_2_1107_5_2_43_167024_2020021412403753831414624_0_0_0.json in session 11472
INFO: Correcting metadata for scan 1_3_12_2_1107_5_2_43_167024_2020021412403753831414624_0_0_0.nii.gz in session 11472
INFO: Renaming acquisition 5e46dbf86dea3129fd2a742f in session 11472
INFO: Correcting metadata for scan 1.3.12.2.1107.5.2.4

INFO: Correcting metadata for scan 1.3.12.2.1107.5.2.43.167024.2020080518503044944436561.0.0.0.dicom.zip in session 11512
INFO: Correcting metadata for scan 1_3_12_2_1107_5_2_43_167024_2020080518503044944436561_0_0_0.json in session 11512
INFO: Correcting metadata for scan 1_3_12_2_1107_5_2_43_167024_2020080518503044944436561_0_0_0.nii.gz in session 11512
INFO: Renaming acquisition 5f2b378d1c2d6b0210028838 in session 11512
INFO: Correcting metadata for scan 1.3.12.2.1107.5.2.43.167024.2020081213593786681233790.0.0.0.dicom.zip in session 11517
INFO: Correcting metadata for scan 1_3_12_2_1107_5_2_43_167024_2020081213593786681233790_0_0_0.json in session 11517
INFO: Correcting metadata for scan 1_3_12_2_1107_5_2_43_167024_2020081213593786681233790_0_0_0.nii.gz in session 11517
INFO: Renaming acquisition 5f342dfa63ca07413822ea42 in session 11517
INFO: Correcting metadata for scan 1.3.12.2.1107.5.2.43.167024.2020082416372041484959841.0.0.0.dicom.zip in session 11521
INFO: Correcting metadat

INFO: Correcting metadata for scan 1_3_12_2_1107_5_2_43_167024_2020110613383955849332831_0_0_0.json in session 11575
INFO: Correcting metadata for scan 1_3_12_2_1107_5_2_43_167024_2020110613383955849332831_0_0_0.nii.gz in session 11575
INFO: Renaming acquisition 5fa597b30c8c12dc1ef358dc in session 11575
INFO: Correcting metadata for scan 1.3.12.2.1107.5.2.43.167024.2020111119255191359853884.0.0.0.dicom.zip in session 11581
INFO: Correcting metadata for scan 1_3_12_2_1107_5_2_43_167024_2020111119255191359853884_0_0_0.json in session 11581
INFO: Correcting metadata for scan 1_3_12_2_1107_5_2_43_167024_2020111119255191359853884_0_0_0.nii.gz in session 11581
INFO: Renaming acquisition 5fac80a5b0511acfaee330cc in session 11581
INFO: Correcting metadata for scan 1.3.12.2.1107.5.2.43.167024.2020111210570547348285483.0.0.0.dicom.zip in session 11582
INFO: Correcting metadata for scan 1_3_12_2_1107_5_2_43_167024_2020111210570547348285483_0_0_0.json in session 11582
INFO: Correcting metadata for

INFO: Correcting metadata for scan 1_3_12_2_1107_5_2_43_167024_2021012512133581612274311_0_0_0.nii.gz in session 11636
INFO: Renaming acquisition 600efcf5069f319c70eac941 in session 11636
INFO: Correcting metadata for scan 1.3.12.2.1107.5.2.43.167024.2021012613142659606337653.0.0.0.dicom.zip in session 11637
INFO: Correcting metadata for scan 1_3_12_2_1107_5_2_43_167024_2021012613142659606337653_0_0_0.json in session 11637
INFO: Correcting metadata for scan 1_3_12_2_1107_5_2_43_167024_2021012613142659606337653_0_0_0.nii.gz in session 11637
INFO: Renaming acquisition 60105cd64303b8970984980b in session 11637
INFO: Correcting metadata for scan 1.3.12.2.1107.5.2.43.167024.2021012713104066175544664.0.0.0.dicom.zip in session 11639
INFO: Correcting metadata for scan 1_3_12_2_1107_5_2_43_167024_2021012713104066175544664_0_0_0.json in session 11639
INFO: Correcting metadata for scan 1_3_12_2_1107_5_2_43_167024_2021012713104066175544664_0_0_0.nii.gz in session 11639
INFO: Renaming acquisition 

INFO: Renaming acquisition 605e095ff48c874c7fd8d694 in session 11700
INFO: Correcting metadata for scan 1.3.12.2.1107.5.2.43.167024.2021040112393616167250727.0.0.0.dicom.zip in session 11710
INFO: Correcting metadata for scan 1_3_12_2_1107_5_2_43_167024_2021040112393616167250727_0_0_0.json in session 11710
INFO: Correcting metadata for scan 1_3_12_2_1107_5_2_43_167024_2021040112393616167250727_0_0_0.nii.gz in session 11710
INFO: Renaming acquisition 6065f8137e7d51e2aac84faa in session 11710
INFO: Correcting metadata for scan 1.3.12.2.1107.5.2.43.167024.2021040213121730839051783.0.0.0.dicom.zip in session 11711
INFO: Correcting metadata for scan 1_3_12_2_1107_5_2_43_167024_2021040213121730839051783_0_0_0.json in session 11711
INFO: Correcting metadata for scan 1_3_12_2_1107_5_2_43_167024_2021040213121730839051783_0_0_0.nii.gz in session 11711
INFO: Renaming acquisition 606751408eaed82683e53ed8 in session 11711
INFO: Correcting metadata for scan 1.3.12.2.1107.5.2.43.167024.20210406131202

INFO: Correcting metadata for scan 1_3_12_2_1107_5_2_43_167024_2021061617213619890948407_0_0_0.json in session 11791
INFO: Correcting metadata for scan 1_3_12_2_1107_5_2_43_167024_2021061617213619890948407_0_0_0.nii.gz in session 11791
INFO: Renaming acquisition 60ca6bf3ba295e18031aaa80 in session 11791
INFO: Correcting metadata for scan 1.3.12.2.1107.5.2.43.167024.2021062517241881322662208.0.0.0.dicom.zip in session 11800
INFO: Correcting metadata for scan 1_3_12_2_1107_5_2_43_167024_2021062517241881322662208_0_0_0.json in session 11800
INFO: Correcting metadata for scan 1_3_12_2_1107_5_2_43_167024_2021062517241881322662208_0_0_0.nii.gz in session 11800
INFO: Renaming acquisition 60d64a247fffa1988a96c669 in session 11800
INFO: Correcting metadata for scan 1.3.12.2.1107.5.2.43.167024.20210629165900295284125.0.0.0.dicom.zip in session 11809
INFO: Correcting metadata for scan 1_3_12_2_1107_5_2_43_167024_20210629165900295284125_0_0_0.json in session 11809
INFO: Correcting metadata for sca

INFO: Correcting metadata for scan 1_3_12_2_1107_5_2_43_167024_2021090314030924944840442_0_0_0.nii.gz in session 11894
INFO: Renaming acquisition 61329af5ec96f053a563dee9 in session 11894
INFO: Correcting metadata for scan 1.3.12.2.1107.5.2.43.167024.2021090813540939980217538.0.0.0.dicom.zip in session 11896
INFO: Correcting metadata for scan 1_3_12_2_1107_5_2_43_167024_2021090813540939980217538_0_0_0.json in session 11896
INFO: Correcting metadata for scan 1_3_12_2_1107_5_2_43_167024_2021090813540939980217538_0_0_0.nii.gz in session 11896
INFO: Renaming acquisition 6138f924628d3a469fe43ae0 in session 11896
INFO: Correcting metadata for scan 1.3.12.2.1107.5.2.43.167024.2021091513592751302850940.0.0.0.dicom.zip in session 11900
INFO: Correcting metadata for scan 1_3_12_2_1107_5_2_43_167024_2021091513592751302850940_0_0_0.json in session 11900
INFO: Correcting metadata for scan 1_3_12_2_1107_5_2_43_167024_2021091513592751302850940_0_0_0.nii.gz in session 11900
INFO: Renaming acquisition 

INFO: Renaming acquisition 61c0d768454dc871dd1bd675 in session 12014
INFO: Correcting metadata for scan 1.3.12.2.1107.5.2.43.167024.2021122114140579042645797.0.0.0.dicom.zip in session 12017
INFO: Correcting metadata for scan 1_3_12_2_1107_5_2_43_167024_2021122114140579042645797_0_0_0.json in session 12017
INFO: Correcting metadata for scan 1_3_12_2_1107_5_2_43_167024_2021122114140579042645797_0_0_0.nii.gz in session 12017
INFO: Renaming acquisition 61c2279ad1cd86a6921bd67f in session 12017
INFO: Correcting metadata for scan 1.3.12.2.1107.5.2.43.167024.2021122214383499867748372.0.0.0.dicom.zip in session 12021
INFO: Correcting metadata for scan 1_3_12_2_1107_5_2_43_167024_2021122214383499867748372_0_0_0.json in session 12021
INFO: Correcting metadata for scan 1_3_12_2_1107_5_2_43_167024_2021122214383499867748372_0_0_0.nii.gz in session 12021
INFO: Renaming acquisition 61c37ecd2340459e762d15a8 in session 12021
INFO: Correcting metadata for scan 1.3.12.2.1107.5.2.43.167024.20220127180415

INFO: Correcting metadata for scan 1_3_12_2_1107_5_2_43_167024_2022051614014633469803546_0_0_0.json in session 12217
INFO: Correcting metadata for scan 1_3_12_2_1107_5_2_43_167024_2022051614014633469803546_0_0_0.nii.gz in session 12217
INFO: Renaming acquisition 6282912374112e16ec1f7495 in session 12217
INFO: Correcting metadata for scan 1.3.12.2.1107.5.2.43.167024.2022062410175354009498673.0.0.0.dicom.zip in session 12243
INFO: Correcting metadata for scan 1_3_12_2_1107_5_2_43_167024_2022062410175354009498673_0_0_0.json in session 12243
INFO: Correcting metadata for scan 1_3_12_2_1107_5_2_43_167024_2022062410175354009498673_0_0_0.nii.gz in session 12243
INFO: Renaming acquisition 62b5c6c9aba61aca70dc54c5 in session 12243
